# Imports

In [70]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
from sklearn.metrics import average_precision_score, roc_auc_score


In [72]:
# --------------------
# Load data
# --------------------
train_df = pd.read_csv(r'sample_loans_train.csv')
valid_df = pd.read_csv(r'sample_loans_valid.csv')

In [74]:
 pd.set_option('display.max_columns', None)
train_df.head()

,index,target,CreditScore,FirstPaymentDate,FirstTimeHomebuyerFlag,MaturityDate,MSA,MI_Pct,NumberOfUnits,OccupancyStatus,OriginalCLTV,OriginalDTI,OriginalUPB,OriginalLTV,OriginalInterestRate,Channel,PPM_Flag,ProductType,PropertyState,PropertyType,PostalCode,LoanPurpose,OriginalLoanTerm,NumberOfBorrowers,SellerName,ServicerName,SuperConformingFlag,PreHARP_Flag,ProgramIndicator,ReliefRefinanceIndicator,PropertyValMethod,InterestOnlyFlag,BalloonIndicator,0_CurrentActualUPB,0_CurrentInterestRate,0_CurrentNonInterestBearingUPB,0_EstimatedLTV,0_InterestBearingUPB,0_LoanAge,0_MonthlyReportingPeriod,0_RemainingMonthsToLegalMaturity,1_CurrentActualUPB,1_CurrentInterestRate,1_CurrentNonInterestBearingUPB,1_EstimatedLTV,1_InterestBearingUPB,1_LoanAge,1_MonthlyReportingPeriod,1_RemainingMonthsToLegalMaturity,2_CurrentActualUPB,2_CurrentInterestRate,2_CurrentNonInterestBearingUPB,2_EstimatedLTV,2_InterestBearingUPB,2_LoanAge,2_MonthlyReportingPeriod,2_RemainingMonthsToLegalMaturity,3_CurrentActualUPB,3_CurrentInterestRate,3_CurrentNonInterestBearingUPB,3_EstimatedLTV,3_InterestBearingUPB,3_LoanAge,3_MonthlyReportingPeriod,3_RemainingMonthsToLegalMaturity,4_CurrentActualUPB,4_CurrentInterestRate,4_CurrentNonInterestBearingUPB,4_EstimatedLTV,4_InterestBearingUPB,4_LoanAge,4_MonthlyReportingPeriod,4_RemainingMonthsToLegalMaturity,5_CurrentActualUPB,5_CurrentInterestRate,5_CurrentNonInterestBearingUPB,5_EstimatedLTV,5_InterestBearingUPB,5_LoanAge,5_MonthlyReportingPeriod,5_RemainingMonthsToLegalMaturity,6_CurrentActualUPB,6_CurrentInterestRate,6_CurrentNonInterestBearingUPB,6_EstimatedLTV,6_InterestBearingUPB,6_LoanAge,6_MonthlyReportingPeriod,6_RemainingMonthsToLegalMaturity,7_CurrentActualUPB,7_CurrentInterestRate,7_CurrentNonInterestBearingUPB,7_EstimatedLTV,7_InterestBearingUPB,7_LoanAge,7_MonthlyReportingPeriod,7_RemainingMonthsToLegalMaturity,8_CurrentActualUPB,8_CurrentInterestRate,8_CurrentNonInterestBearingUPB,8_EstimatedLTV,8_InterestBearingUPB,8_LoanAge,8_MonthlyReportingPeriod,8_RemainingMonthsToLegalMaturity,9_CurrentActualUPB,9_CurrentInterestRate,9_CurrentNonInterestBearingUPB,9_EstimatedLTV,9_InterestBearingUPB,9_LoanAge,9_MonthlyReportingPeriod,9_RemainingMonthsToLegalMaturity,10_CurrentActualUPB,10_CurrentInterestRate,10_CurrentNonInterestBearingUPB,10_EstimatedLTV,10_InterestBearingUPB,10_LoanAge,10_MonthlyReportingPeriod,10_RemainingMonthsToLegalMaturity,11_CurrentActualUPB,11_CurrentInterestRate,11_CurrentNonInterestBearingUPB,11_EstimatedLTV,11_InterestBearingUPB,11_LoanAge,11_MonthlyReportingPeriod,11_RemainingMonthsToLegalMaturity,12_CurrentActualUPB,12_CurrentInterestRate,12_CurrentNonInterestBearingUPB,12_EstimatedLTV,12_InterestBearingUPB,12_LoanAge,12_MonthlyReportingPeriod,12_RemainingMonthsToLegalMaturity,13_CurrentActualUPB,13_CurrentInterestRate,13_CurrentNonInterestBearingUPB,13_EstimatedLTV,13_InterestBearingUPB,13_LoanAge,13_MonthlyReportingPeriod,13_RemainingMonthsToLegalMaturity
0,0,0,747,202403,N,205402,NaN,0,1,P,67,26,82000,67,8.000,R,N,FRM,KS,MH,67600,P,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7,82000.0,8.000,0.0,999,82000.0,0,202402,360,82000.0,8.000,0.0,999,82000.0,1,202403,359,82000.0,8.000,0.0,999,82000.0,2,202404,358,82000.0,8.000,0.0,999,82000.0,3,202405,357,82000.0,8.000,0.0,999,82000.0,4,202406,356,82000.0,8.000,0.0,999,82000.0,5,202407,355,82000.0,8.000,0.0,999,82000.0,6,202408,354,81546.88,8.000,0.0,999,81546.88,7,202409,353,81488.53,8.000,0.0,999,81488.53,8,202410,352,81429.79,8.000,0.0,999,81429.79,9,202411,351,81370.66,8.000,0.0,999,81370.66,10,202412,350,81311.13,8.000,0.0,999,81311.13,11,202501,349,81251.20,8.000,0.0,999,81251.20,12,202502,348,81190.87,8.000,0.0,999,81190.87,13,202503,347
1,1,0,659,202403,N,205402,NaN,0,1,P,68,32,305000,68,7.875,R,N,FRM,MI,SF,49000,P,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7,305000.0,7.875,0.0,67,305000.0,0,202402,360,305000.0,7.875,0.0,66,305000.0,1,202403,359,304000.0,7.875,0.0,64,304000.0,2,202404,358,304000.0,7.875,0.0,63,3040

In [80]:
train_df[train_df.target==0].head(1)

,index,target,CreditScore,FirstPaymentDate,FirstTimeHomebuyerFlag,MaturityDate,MSA,MI_Pct,NumberOfUnits,OccupancyStatus,OriginalCLTV,OriginalDTI,OriginalUPB,OriginalLTV,OriginalInterestRate,Channel,PPM_Flag,ProductType,PropertyState,PropertyType,PostalCode,LoanPurpose,OriginalLoanTerm,NumberOfBorrowers,SellerName,ServicerName,SuperConformingFlag,PreHARP_Flag,ProgramIndicator,ReliefRefinanceIndicator,PropertyValMethod,InterestOnlyFlag,BalloonIndicator,0_CurrentActualUPB,0_CurrentInterestRate,0_CurrentNonInterestBearingUPB,0_EstimatedLTV,0_InterestBearingUPB,0_LoanAge,0_MonthlyReportingPeriod,0_RemainingMonthsToLegalMaturity,1_CurrentActualUPB,1_CurrentInterestRate,1_CurrentNonInterestBearingUPB,1_EstimatedLTV,1_InterestBearingUPB,1_LoanAge,1_MonthlyReportingPeriod,1_RemainingMonthsToLegalMaturity,2_CurrentActualUPB,2_CurrentInterestRate,2_CurrentNonInterestBearingUPB,2_EstimatedLTV,2_InterestBearingUPB,2_LoanAge,2_MonthlyReportingPeriod,2_RemainingMonthsToLegalMaturity,3_CurrentActualUPB,3_CurrentInterestRate,3_CurrentNonInterestBearingUPB,3_EstimatedLTV,3_InterestBearingUPB,3_LoanAge,3_MonthlyReportingPeriod,3_RemainingMonthsToLegalMaturity,4_CurrentActualUPB,4_CurrentInterestRate,4_CurrentNonInterestBearingUPB,4_EstimatedLTV,4_InterestBearingUPB,4_LoanAge,4_MonthlyReportingPeriod,4_RemainingMonthsToLegalMaturity,5_CurrentActualUPB,5_CurrentInterestRate,5_CurrentNonInterestBearingUPB,5_EstimatedLTV,5_InterestBearingUPB,5_LoanAge,5_MonthlyReportingPeriod,5_RemainingMonthsToLegalMaturity,6_CurrentActualUPB,6_CurrentInterestRate,6_CurrentNonInterestBearingUPB,6_EstimatedLTV,6_InterestBearingUPB,6_LoanAge,6_MonthlyReportingPeriod,6_RemainingMonthsToLegalMaturity,7_CurrentActualUPB,7_CurrentInterestRate,7_CurrentNonInterestBearingUPB,7_EstimatedLTV,7_InterestBearingUPB,7_LoanAge,7_MonthlyReportingPeriod,7_RemainingMonthsToLegalMaturity,8_CurrentActualUPB,8_CurrentInterestRate,8_CurrentNonInterestBearingUPB,8_EstimatedLTV,8_InterestBearingUPB,8_LoanAge,8_MonthlyReportingPeriod,8_RemainingMonthsToLegalMaturity,9_CurrentActualUPB,9_CurrentInterestRate,9_CurrentNonInterestBearingUPB,9_EstimatedLTV,9_InterestBearingUPB,9_LoanAge,9_MonthlyReportingPeriod,9_RemainingMonthsToLegalMaturity,10_CurrentActualUPB,10_CurrentInterestRate,10_CurrentNonInterestBearingUPB,10_EstimatedLTV,10_InterestBearingUPB,10_LoanAge,10_MonthlyReportingPeriod,10_RemainingMonthsToLegalMaturity,11_CurrentActualUPB,11_CurrentInterestRate,11_CurrentNonInterestBearingUPB,11_EstimatedLTV,11_InterestBearingUPB,11_LoanAge,11_MonthlyReportingPeriod,11_RemainingMonthsToLegalMaturity,12_CurrentActualUPB,12_CurrentInterestRate,12_CurrentNonInterestBearingUPB,12_EstimatedLTV,12_InterestBearingUPB,12_LoanAge,12_MonthlyReportingPeriod,12_RemainingMonthsToLegalMaturity,13_CurrentActualUPB,13_CurrentInterestRate,13_CurrentNonInterestBearingUPB,13_EstimatedLTV,13_InterestBearingUPB,13_LoanAge,13_MonthlyReportingPeriod,13_RemainingMonthsToLegalMaturity
0,0,0,747,202403,N,205402,NaN,0,1,P,67,26,82000,67,8.0,R,N,FRM,KS,MH,67600,P,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7,82000.0,8.0,0.0,999,82000.0,0,202402,360,82000.0,8.0,0.0,999,82000.0,1,202403,359,82000.0,8.0,0.0,999,82000.0,2,202404,358,82000.0,8.0,0.0,999,82000.0,3,202405,357,82000.0,8.0,0.0,999,82000.0,4,202406,356,82000.0,8.0,0.0,999,82000.0,5,202407,355,82000.0,8.0,0.0,999,82000.0,6,202408,354,81546.88,8.0,0.0,999,81546.88,7,202409,353,81488.53,8.0,0.0,999,81488.53,8,202410,352,81429.79,8.0,0.0,999,81429.79,9,202411,351,81370.66,8.0,0.0,999,81370.66,10,202412,350,81311.13,8.0,0.0,999,81311.13,11,202501,349,81251.2,8.0,0.0,999,81251.2,12,202502,348,81190.87,8.0,0.0,999,81190.87,13,202503,347


In [82]:
valid_df[valid_df.target==1].head(1)

,index,target,CreditScore,FirstPaymentDate,FirstTimeHomebuyerFlag,MaturityDate,MSA,MI_Pct,NumberOfUnits,OccupancyStatus,OriginalCLTV,OriginalDTI,OriginalUPB,OriginalLTV,OriginalInterestRate,Channel,PPM_Flag,ProductType,PropertyState,PropertyType,PostalCode,LoanPurpose,OriginalLoanTerm,NumberOfBorrowers,SellerName,ServicerName,SuperConformingFlag,PreHARP_Flag,ProgramIndicator,ReliefRefinanceIndicator,PropertyValMethod,InterestOnlyFlag,BalloonIndicator,0_CurrentActualUPB,0_CurrentInterestRate,0_CurrentNonInterestBearingUPB,0_EstimatedLTV,0_InterestBearingUPB,0_LoanAge,0_MonthlyReportingPeriod,0_RemainingMonthsToLegalMaturity,1_CurrentActualUPB,1_CurrentInterestRate,1_CurrentNonInterestBearingUPB,1_EstimatedLTV,1_InterestBearingUPB,1_LoanAge,1_MonthlyReportingPeriod,1_RemainingMonthsToLegalMaturity,2_CurrentActualUPB,2_CurrentInterestRate,2_CurrentNonInterestBearingUPB,2_EstimatedLTV,2_InterestBearingUPB,2_LoanAge,2_MonthlyReportingPeriod,2_RemainingMonthsToLegalMaturity,3_CurrentActualUPB,3_CurrentInterestRate,3_CurrentNonInterestBearingUPB,3_EstimatedLTV,3_InterestBearingUPB,3_LoanAge,3_MonthlyReportingPeriod,3_RemainingMonthsToLegalMaturity,4_CurrentActualUPB,4_CurrentInterestRate,4_CurrentNonInterestBearingUPB,4_EstimatedLTV,4_InterestBearingUPB,4_LoanAge,4_MonthlyReportingPeriod,4_RemainingMonthsToLegalMaturity,5_CurrentActualUPB,5_CurrentInterestRate,5_CurrentNonInterestBearingUPB,5_EstimatedLTV,5_InterestBearingUPB,5_LoanAge,5_MonthlyReportingPeriod,5_RemainingMonthsToLegalMaturity,6_CurrentActualUPB,6_CurrentInterestRate,6_CurrentNonInterestBearingUPB,6_EstimatedLTV,6_InterestBearingUPB,6_LoanAge,6_MonthlyReportingPeriod,6_RemainingMonthsToLegalMaturity,7_CurrentActualUPB,7_CurrentInterestRate,7_CurrentNonInterestBearingUPB,7_EstimatedLTV,7_InterestBearingUPB,7_LoanAge,7_MonthlyReportingPeriod,7_RemainingMonthsToLegalMaturity,8_CurrentActualUPB,8_CurrentInterestRate,8_CurrentNonInterestBearingUPB,8_EstimatedLTV,8_InterestBearingUPB,8_LoanAge,8_MonthlyReportingPeriod,8_RemainingMonthsToLegalMaturity,9_CurrentActualUPB,9_CurrentInterestRate,9_CurrentNonInterestBearingUPB,9_EstimatedLTV,9_InterestBearingUPB,9_LoanAge,9_MonthlyReportingPeriod,9_RemainingMonthsToLegalMaturity,10_CurrentActualUPB,10_CurrentInterestRate,10_CurrentNonInterestBearingUPB,10_EstimatedLTV,10_InterestBearingUPB,10_LoanAge,10_MonthlyReportingPeriod,10_RemainingMonthsToLegalMaturity,11_CurrentActualUPB,11_CurrentInterestRate,11_CurrentNonInterestBearingUPB,11_EstimatedLTV,11_InterestBearingUPB,11_LoanAge,11_MonthlyReportingPeriod,11_RemainingMonthsToLegalMaturity,12_CurrentActualUPB,12_CurrentInterestRate,12_CurrentNonInterestBearingUPB,12_EstimatedLTV,12_InterestBearingUPB,12_LoanAge,12_MonthlyReportingPeriod,12_RemainingMonthsToLegalMaturity,13_CurrentActualUPB,13_CurrentInterestRate,13_CurrentNonInterestBearingUPB,13_EstimatedLTV,13_InterestBearingUPB,13_LoanAge,13_MonthlyReportingPeriod,13_RemainingMonthsToLegalMaturity
13,124,1,676,202403,Y,205402,35380.0,0,1,P,73,49,197000,73,6.875,R,N,FRM,LA,PU,70000,P,360,2,Other sellers,NATIONSTAR MORTGAGE LLC DBA MR. COOPER,NaN,NaN,9,NaN,2,N,7,197000.0,6.875,0.0,74,197000.0,0,202402,360,197000.0,6.875,0.0,71,197000.0,1,202403,359,197000.0,6.875,0.0,70,197000.0,2,202404,358,197000.0,6.875,0.0,70,197000.0,3,202405,357,197000.0,6.875,0.0,70,197000.0,4,202406,356,196000.0,6.875,0.0,71,196000.0,5,202407,355,196000.0,6.875,0.0,72,196000.0,6,202408,354,196089.77,6.875,0.0,72,196089.77,7,202409,353,195917.28,6.875,0.0,72,195917.28,8,202410,352,195743.8,6.875,0.0,72,195743.8,9,202411,351,195569.33,6.875,0.0,71,195569.33,10,202412,350,195393.86,6.875,0.0,70,195393.86,11,202501,349,195217.38,6.875,0.0,71,195217.38,12,202502,348,195217.38,6.875,0.0,72,195217.38,13,202503,347


In [84]:
valid_df.describe()

,index,target,CreditScore,FirstPaymentDate,MaturityDate,MSA,MI_Pct,NumberOfUnits,OriginalCLTV,OriginalDTI,OriginalUPB,OriginalLTV,OriginalInterestRate,PostalCode,OriginalLoanTerm,NumberOfBorrowers,PreHARP_Flag,ReliefRefinanceIndicator,PropertyValMethod,0_CurrentActualUPB,0_CurrentInterestRate,0_CurrentNonInterestBearingUPB,0_EstimatedLTV,0_InterestBearingUPB,0_LoanAge,0_MonthlyReportingPeriod,0_RemainingMonthsToLegalMaturity,1_CurrentActualUPB,1_CurrentInterestRate,1_CurrentNonInterestBearingUPB,1_EstimatedLTV,1_InterestBearingUPB,1_LoanAge,1_MonthlyReportingPeriod,1_RemainingMonthsToLegalMaturity,2_CurrentActualUPB,2_CurrentInterestRate,2_CurrentNonInterestBearingUPB,2_EstimatedLTV,2_InterestBearingUPB,2_LoanAge,2_MonthlyReportingPeriod,2_RemainingMonthsToLegalMaturity,3_CurrentActualUPB,3_CurrentInterestRate,3_CurrentNonInterestBearingUPB,3_EstimatedLTV,3_InterestBearingUPB,3_LoanAge,3_MonthlyReportingPeriod,3_RemainingMonthsToLegalMaturity,4_CurrentActualUPB,4_CurrentInterestRate,4_CurrentNonInterestBearingUPB,4_EstimatedLTV,4_InterestBearingUPB,4_LoanAge,4_MonthlyReportingPeriod,4_RemainingMonthsToLegalMaturity,5_CurrentActualUPB,5_CurrentInterestRate,5_CurrentNonInterestBearingUPB,5_EstimatedLTV,5_InterestBearingUPB,5_LoanAge,5_MonthlyReportingPeriod,5_RemainingMonthsToLegalMaturity,6_CurrentActualUPB,6_CurrentInterestRate,6_CurrentNonInterestBearingUPB,6_EstimatedLTV,6_InterestBearingUPB,6_LoanAge,6_MonthlyReportingPeriod,6_RemainingMonthsToLegalMaturity,7_CurrentActualUPB,7_CurrentInterestRate,7_CurrentNonInterestBearingUPB,7_EstimatedLTV,7_InterestBearingUPB,7_LoanAge,7_MonthlyReportingPeriod,7_RemainingMonthsToLegalMaturity,8_CurrentActualUPB,8_CurrentInterestRate,8_CurrentNonInterestBearingUPB,8_EstimatedLTV,8_InterestBearingUPB,8_LoanAge,8_MonthlyReportingPeriod,8_RemainingMonthsToLegalMaturity,9_CurrentActualUPB,9_CurrentInterestRate,9_CurrentNonInterestBearingUPB,9_EstimatedLTV,9_InterestBearingUPB,9_LoanAge,9_MonthlyReportingPeriod,9_RemainingMonthsToLegalMaturity,10_CurrentActualUPB,10_CurrentInterestRate,10_CurrentNonInterestBearingUPB,10_EstimatedLTV,10_InterestBearingUPB,10_LoanAge,10_MonthlyReportingPeriod,10_RemainingMonthsToLegalMaturity,11_CurrentActualUPB,11_CurrentInterestRate,11_CurrentNonInterestBearingUPB,11_EstimatedLTV,11_InterestBearingUPB,11_LoanAge,11_MonthlyReportingPeriod,11_RemainingMonthsToLegalMaturity,12_CurrentActualUPB,12_CurrentInterestRate,12_CurrentNonInterestBearingUPB,12_EstimatedLTV,12_InterestBearingUPB,12_LoanAge,12_MonthlyReportingPeriod,12_RemainingMonthsToLegalMaturity,13_CurrentActualUPB,13_CurrentInterestRate,13_CurrentNonInterestBearingUPB,13_EstimatedLTV,13_InterestBearingUPB,13_LoanAge,13_MonthlyReportingPeriod,13_RemainingMonthsToLegalMaturity
count,1075.000000,1075.000000,1075.000000,1075.000000,1075.000000,903.000000,1075.000000,1075.000000,1075.000000,1075.000000,1.075000e+03,1075.000000,1075.000000,1075.000000,1075.000000,1075.000000,0.0,0.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.0,1075.000000,1.075000e+03,1075.000000,1075.000000,1075.000000,1.075000e+03,107

In [86]:
train_df.describe()

,index,target,CreditScore,FirstPaymentDate,MaturityDate,MSA,MI_Pct,NumberOfUnits,OriginalCLTV,OriginalDTI,OriginalUPB,OriginalLTV,OriginalInterestRate,PostalCode,OriginalLoanTerm,NumberOfBorrowers,PreHARP_Flag,ReliefRefinanceIndicator,PropertyValMethod,0_CurrentActualUPB,0_CurrentInterestRate,0_CurrentNonInterestBearingUPB,0_EstimatedLTV,0_InterestBearingUPB,0_LoanAge,0_MonthlyReportingPeriod,0_RemainingMonthsToLegalMaturity,1_CurrentActualUPB,1_CurrentInterestRate,1_CurrentNonInterestBearingUPB,1_EstimatedLTV,1_InterestBearingUPB,1_LoanAge,1_MonthlyReportingPeriod,1_RemainingMonthsToLegalMaturity,2_CurrentActualUPB,2_CurrentInterestRate,2_CurrentNonInterestBearingUPB,2_EstimatedLTV,2_InterestBearingUPB,2_LoanAge,2_MonthlyReportingPeriod,2_RemainingMonthsToLegalMaturity,3_CurrentActualUPB,3_CurrentInterestRate,3_CurrentNonInterestBearingUPB,3_EstimatedLTV,3_InterestBearingUPB,3_LoanAge,3_MonthlyReportingPeriod,3_RemainingMonthsToLegalMaturity,4_CurrentActualUPB,4_CurrentInterestRate,4_CurrentNonInterestBearingUPB,4_EstimatedLTV,4_InterestBearingUPB,4_LoanAge,4_MonthlyReportingPeriod,4_RemainingMonthsToLegalMaturity,5_CurrentActualUPB,5_CurrentInterestRate,5_CurrentNonInterestBearingUPB,5_EstimatedLTV,5_InterestBearingUPB,5_LoanAge,5_MonthlyReportingPeriod,5_RemainingMonthsToLegalMaturity,6_CurrentActualUPB,6_CurrentInterestRate,6_CurrentNonInterestBearingUPB,6_EstimatedLTV,6_InterestBearingUPB,6_LoanAge,6_MonthlyReportingPeriod,6_RemainingMonthsToLegalMaturity,7_CurrentActualUPB,7_CurrentInterestRate,7_CurrentNonInterestBearingUPB,7_EstimatedLTV,7_InterestBearingUPB,7_LoanAge,7_MonthlyReportingPeriod,7_RemainingMonthsToLegalMaturity,8_CurrentActualUPB,8_CurrentInterestRate,8_CurrentNonInterestBearingUPB,8_EstimatedLTV,8_InterestBearingUPB,8_LoanAge,8_MonthlyReportingPeriod,8_RemainingMonthsToLegalMaturity,9_CurrentActualUPB,9_CurrentInterestRate,9_CurrentNonInterestBearingUPB,9_EstimatedLTV,9_InterestBearingUPB,9_LoanAge,9_MonthlyReportingPeriod,9_RemainingMonthsToLegalMaturity,10_CurrentActualUPB,10_CurrentInterestRate,10_CurrentNonInterestBearingUPB,10_EstimatedLTV,10_InterestBearingUPB,10_LoanAge,10_MonthlyReportingPeriod,10_RemainingMonthsToLegalMaturity,11_CurrentActualUPB,11_CurrentInterestRate,11_CurrentNonInterestBearingUPB,11_EstimatedLTV,11_InterestBearingUPB,11_LoanAge,11_MonthlyReportingPeriod,11_RemainingMonthsToLegalMaturity,12_CurrentActualUPB,12_CurrentInterestRate,12_CurrentNonInterestBearingUPB,12_EstimatedLTV,12_InterestBearingUPB,12_LoanAge,12_MonthlyReportingPeriod,12_RemainingMonthsToLegalMaturity,13_CurrentActualUPB,13_CurrentInterestRate,13_CurrentNonInterestBearingUPB,13_EstimatedLTV,13_InterestBearingUPB,13_LoanAge,13_MonthlyReportingPeriod,13_RemainingMonthsToLegalMaturity
count,6101.000000,6101.0,6101.000000,6101.000000,6101.000000,5230.000000,6101.000000,6101.000000,6101.000000,6101.000000,6.101000e+03,6101.000000,6101.000000,6101.000000,6101.000000,6101.000000,0.0,0.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,6101.0,6101.000000,6.101000e+03,6101.000000,

In [88]:
panel_cols = [c for c in train_df.columns if "_" in c and any(c.startswith(f"{i}_") for i in range(14))]

In [90]:
panel_cols

['0_CurrentActualUPB',
 '0_CurrentInterestRate',
 '0_CurrentNonInterestBearingUPB',
 '0_EstimatedLTV',
 '0_InterestBearingUPB',
 '0_LoanAge',
 '0_MonthlyReportingPeriod',
 '0_RemainingMonthsToLegalMaturity',
 '1_CurrentActualUPB',
 '1_CurrentInterestRate',
 '1_CurrentNonInterestBearingUPB',
 '1_EstimatedLTV',
 '1_InterestBearingUPB',
 '1_LoanAge',
 '1_MonthlyReportingPeriod',
 '1_RemainingMonthsToLegalMaturity',
 '2_CurrentActualUPB',
 '2_CurrentInterestRate',
 '2_CurrentNonInterestBearingUPB',
 '2_EstimatedLTV',
 '2_InterestBearingUPB',
 '2_LoanAge',
 '2_MonthlyReportingPeriod',
 '2_RemainingMonthsToLegalMaturity',
 '3_CurrentActualUPB',
 '3_CurrentInterestRate',
 '3_CurrentNonInterestBearingUPB',
 '3_EstimatedLTV',
 '3_InterestBearingUPB',
 '3_LoanAge',
 '3_MonthlyReportingPeriod',
 '3_RemainingMonthsToLegalMaturity',
 '4_CurrentActualUPB',
 '4_CurrentInterestRate',
 '4_CurrentNonInterestBearingUPB',
 '4_EstimatedLTV',
 '4_InterestBearingUPB',
 '4_LoanAge',
 '4_MonthlyReportingPeriod

In [98]:
list(train_df.columns)

['index',
 'target',
 'CreditScore',
 'FirstPaymentDate',
 'FirstTimeHomebuyerFlag',
 'MaturityDate',
 'MSA',
 'MI_Pct',
 'NumberOfUnits',
 'OccupancyStatus',
 'OriginalCLTV',
 'OriginalDTI',
 'OriginalUPB',
 'OriginalLTV',
 'OriginalInterestRate',
 'Channel',
 'PPM_Flag',
 'ProductType',
 'PropertyState',
 'PropertyType',
 'PostalCode',
 'LoanPurpose',
 'OriginalLoanTerm',
 'NumberOfBorrowers',
 'SellerName',
 'ServicerName',
 'SuperConformingFlag',
 'PreHARP_Flag',
 'ProgramIndicator',
 'ReliefRefinanceIndicator',
 'PropertyValMethod',
 'InterestOnlyFlag',
 'BalloonIndicator',
 '0_CurrentActualUPB',
 '0_CurrentInterestRate',
 '0_CurrentNonInterestBearingUPB',
 '0_EstimatedLTV',
 '0_InterestBearingUPB',
 '0_LoanAge',
 '0_MonthlyReportingPeriod',
 '0_RemainingMonthsToLegalMaturity',
 '1_CurrentActualUPB',
 '1_CurrentInterestRate',
 '1_CurrentNonInterestBearingUPB',
 '1_EstimatedLTV',
 '1_InterestBearingUPB',
 '1_LoanAge',
 '1_MonthlyReportingPeriod',
 '1_RemainingMonthsToLegalMaturity'

In [92]:
static_cols = [c for c in train_df.columns if c not in panel_cols]

In [94]:
static_cols

['index',
 'target',
 'CreditScore',
 'FirstPaymentDate',
 'FirstTimeHomebuyerFlag',
 'MaturityDate',
 'MSA',
 'MI_Pct',
 'NumberOfUnits',
 'OccupancyStatus',
 'OriginalCLTV',
 'OriginalDTI',
 'OriginalUPB',
 'OriginalLTV',
 'OriginalInterestRate',
 'Channel',
 'PPM_Flag',
 'ProductType',
 'PropertyState',
 'PropertyType',
 'PostalCode',
 'LoanPurpose',
 'OriginalLoanTerm',
 'NumberOfBorrowers',
 'SellerName',
 'ServicerName',
 'SuperConformingFlag',
 'PreHARP_Flag',
 'ProgramIndicator',
 'ReliefRefinanceIndicator',
 'PropertyValMethod',
 'InterestOnlyFlag',
 'BalloonIndicator']

In [108]:
lst = []
for i in range(14):
    lst.append(f"{i}_MonthlyReportingPeriod")

In [112]:
lst.extend(["FirstPaymentDate", "MaturityDate", "index"])


In [114]:
categorical_cols = [
    "FirstTimeHomebuyerFlag","MSA", "OccupancyStatus", "Channel",
    "PPM_Flag", "ProductType", "PropertyState", "PropertyType", "PostalCode",
    "LoanPurpose","SellerName", "ServicerName",
    "SuperConformingFlag", "PreHARP_Flag", "ProgramIndicator",
    "ReliefRefinanceIndicator", "PropertyValMethod",
    "InterestOnlyFlag", "BalloonIndicator"
]
numeric_cols = [c for c in X_train.columns if c not in categorical_cols and c not in lst]

In [116]:
numeric_cols

['index',
 'CreditScore',
 'MI_Pct',
 'NumberOfUnits',
 'OriginalCLTV',
 'OriginalDTI',
 'OriginalUPB',
 'OriginalLTV',
 'OriginalInterestRate',
 'OriginalLoanTerm',
 'NumberOfBorrowers',
 '0_CurrentActualUPB',
 '0_CurrentInterestRate',
 '0_CurrentNonInterestBearingUPB',
 '0_EstimatedLTV',
 '0_InterestBearingUPB',
 '0_LoanAge',
 '0_RemainingMonthsToLegalMaturity',
 '1_CurrentActualUPB',
 '1_CurrentInterestRate',
 '1_CurrentNonInterestBearingUPB',
 '1_EstimatedLTV',
 '1_InterestBearingUPB',
 '1_LoanAge',
 '1_RemainingMonthsToLegalMaturity',
 '2_CurrentActualUPB',
 '2_CurrentInterestRate',
 '2_CurrentNonInterestBearingUPB',
 '2_EstimatedLTV',
 '2_InterestBearingUPB',
 '2_LoanAge',
 '2_RemainingMonthsToLegalMaturity',
 '3_CurrentActualUPB',
 '3_CurrentInterestRate',
 '3_CurrentNonInterestBearingUPB',
 '3_EstimatedLTV',
 '3_InterestBearingUPB',
 '3_LoanAge',
 '3_RemainingMonthsToLegalMaturity',
 '4_CurrentActualUPB',
 '4_CurrentInterestRate',
 '4_CurrentNonInterestBearingUPB',
 '4_Estimate

In [130]:
process_ids_valid = valid_df["index"].values
X_train_feat = X_train[categorical_cols + numeric_cols]
X_valid_feat = X_valid[categorical_cols + numeric_cols]

In [132]:
X_train = train_df.drop(columns=["target"])
X_valid = valid_df.drop(columns=["target"])
y_valid = valid_df["target"].values

# categorical_cols = ["processName", "eventName", "hostName"]
# numeric_cols = ["argsNum", "returnValue", "userId", "parentProcessId", "threadId"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), categorical_cols),
        ("num", StandardScaler(with_mean=False), numeric_cols),
    ],
    sparse_threshold=1.0, 
)

# --------------------
# Baseline Model: Isolation Forest
# --------------------
model = IsolationForest(
    n_estimators=200,
    max_samples="auto",
    contamination="auto",
    random_state=42,
    n_jobs=-1,
)

pipe = Pipeline([("prep", preprocess), ("clf", model)])

# Train on normal only
pipe.fit(X_train_feat)

# Score validation
# IsolationForest: higher score_samples = more normal
scores_normal = pipe["clf"].score_samples(pipe["prep"].transform(X_valid_feat))
raw_anom = -scores_normal  # invert: higher = more anomalous

# Normalize to [0,1]
min_v, max_v = np.min(raw_anom), np.max(raw_anom)
anom_score = (raw_anom - min_v) / (max_v - min_v + 1e-12)

# --------------------
# Evaluation
# --------------------
ap = average_precision_score(y_valid, anom_score)
roc_auc = roc_auc_score(y_valid, anom_score)

print(f"AP: {ap:.4f}")
print(f"AUC-ROC : {roc_auc:.4f}")

# Sample Output CSV
out_df = pd.DataFrame({"index": process_ids_valid, "anomaly_score": anom_score})
out_df.to_csv("valid_pred.csv", index=False)
print("Wrote valid_pred.csv")

AP: 0.1424
AUC-ROC : 0.5159
Wrote valid_pred.csv


In [134]:
from sklearn.svm import OneClassSVM

model = OneClassSVM(
    kernel="rbf",     # RBF works well for non-linear anomalies
    nu=0.05,          # Upper bound on fraction of anomalies in training set
    gamma="scale"     # Kernel coefficient (can tune)
)


pipe = Pipeline([("prep", preprocess), ("clf", model)])

# Train on normal only
pipe.fit(X_train_feat)

# Score validation
# OneClassSVM: higher score_samples = more normal
scores_normal = pipe["clf"].score_samples(pipe["prep"].transform(X_valid_feat))
raw_anom = -scores_normal  # invert: higher = more anomalous

# Normalize to [0,1]
min_v, max_v = np.min(raw_anom), np.max(raw_anom)
anom_score = (raw_anom - min_v) / (max_v - min_v + 1e-12)

# --------------------
# Evaluation
# --------------------
ap = average_precision_score(y_valid, anom_score)
roc_auc = roc_auc_score(y_valid, anom_score)

print(f"AP: {ap:.4f}")
print(f"AUC-ROC : {roc_auc:.4f}")

# Sample Output CSV
out_df = pd.DataFrame({"index": process_ids_valid, "anomaly_score": anom_score})
out_df.to_csv("valid_pred.csv", index=False)
print("Wrote valid_pred.csv")

AP: 0.1638
AUC-ROC : 0.5590


PermissionError: [Errno 13] Permission denied: 'valid_pred.csv'

In [58]:
train_df.columns[0:50]

Index(['index', 'target', 'CreditScore', 'FirstPaymentDate',
       'FirstTimeHomebuyerFlag', 'MaturityDate', 'MSA', 'MI_Pct',
       'NumberOfUnits', 'OccupancyStatus', 'OriginalCLTV', 'OriginalDTI',
       'OriginalUPB', 'OriginalLTV', 'OriginalInterestRate', 'Channel',
       'PPM_Flag', 'ProductType', 'PropertyState', 'PropertyType',
       'PostalCode', 'LoanPurpose', 'OriginalLoanTerm', 'NumberOfBorrowers',
       'SellerName', 'ServicerName', 'SuperConformingFlag', 'PreHARP_Flag',
       'ProgramIndicator', 'ReliefRefinanceIndicator', 'PropertyValMethod',
       'InterestOnlyFlag', 'BalloonIndicator', '0_CurrentActualUPB',
       '0_CurrentInterestRate', '0_CurrentNonInterestBearingUPB',
       '0_EstimatedLTV', '0_InterestBearingUPB', '0_LoanAge',
       '0_MonthlyReportingPeriod', '0_RemainingMonthsToLegalMaturity',
       '1_CurrentActualUPB', '1_CurrentInterestRate',
       '1_CurrentNonInterestBearingUPB', '1_EstimatedLTV',
       '1_InterestBearingUPB', '1_LoanAge', '1_Mon

In [ ]:
static_cols = [
    "CreditScore","FirstPaymentDate", "MaturityDate", "MSA", "MI_Pct" "OriginalUPB", "OriginalLTV", "OriginalDTI", "OriginalInterestRate",
    "OriginalLoanTerm", "NumberOfBorrowers", "NumberOfUnits"
    # add other origination features you want
]

In [ ]:
process_ids_valid = valid_df["index"].values
X_train = train_df.drop(columns=["target"])
X_valid = valid_df.drop(columns=["target"])

y_valid = valid_df["target"].values

# --------------------
# Preprocessing & Feature engineering
# --------------------
categorical_cols = ["processName", "eventName", "hostName"]
numeric_cols = ["argsNum", "returnValue", "userId", "parentProcessId", "threadId"]

X_train_feat = X_train[categorical_cols + numeric_cols]
X_valid_feat = X_valid[categorical_cols + numeric_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), categorical_cols),
        ("num", StandardScaler(with_mean=False), numeric_cols),
    ],
    sparse_threshold=1.0, 
)

# --------------------
# Baseline Model: Isolation Forest
# --------------------
model = IsolationForest(
    n_estimators=200,
    max_samples="auto",
    contamination="auto",
    random_state=42,
    n_jobs=-1,
)

pipe = Pipeline([("prep", preprocess), ("clf", model)])

# Train on normal only
pipe.fit(X_train_feat)

# Score validation
# IsolationForest: higher score_samples = more normal
scores_normal = pipe["clf"].score_samples(pipe["prep"].transform(X_valid_feat))
raw_anom = -scores_normal  # invert: higher = more anomalous

# Normalize to [0,1]
min_v, max_v = np.min(raw_anom), np.max(raw_anom)
anom_score = (raw_anom - min_v) / (max_v - min_v + 1e-12)

# --------------------
# Evaluation
# --------------------
ap = average_precision_score(y_valid, anom_score)
roc_auc = roc_auc_score(y_valid, anom_score)

print(f"AP: {ap:.4f}")
print(f"AUC-ROC : {roc_auc:.4f}")

# Sample Output CSV
out_df = pd.DataFrame({"index": process_ids_valid, "anomaly_score": anom_score})
out_df.to_csv("valid_pred.csv", index=False)
print("Wrote valid_pred.csv")
